# Q9. Stream Algorithm for clustering

In [0]:
# Importing Required libraries
import numpy as np # numpy for array manipulation
import random as rd # random to generete random numbers
from sklearn.datasets import load_iris # load_iris to load iris dataset

# Importing DataSet

In [0]:
data_X,y = load_iris(return_X_y=True) # Loading boston dataset
n = data_X.shape[1] # No. of Features Example

# Initiallizing Hyperparameter

In [0]:
#Hyperparameter
n_iter = 100 # Fix number of iteration
K = 3 # No. of clusters we want to create
ms = int(data_X.shape[0]/3) #chunk size

# Function to Calculate Weighted K-median

In [0]:
def k_med(X,K,n_iter=100,weights=[]):

    m = X.shape[0] # No. of training Examples
    n = X.shape[1] # No. of Features Examples

    # if length of weights == 0  meaning these are the original data points, and hence for that weight of each original data point is 1
    # but when the lebvel 1 representatives would be passed, their weights would not be null...
    if(len(weights) == 0):   
        weights = np.ones(m,dtype = np.int8)  

    #print(weights)

    Centroids=np.array([]).reshape(n,0) # Initiallizing centriod for each feature

    for i in range(K): # Logic for selecting centroid randomly intitally
        rand = rd.randint(0,m-1) # Any random number between 0 to m-1
        Centroids = np.c_[Centroids,X[rand]] # Adding odd centriod as a column in the new one

    EuDis = np.array([]).reshape(m,0) # Initiallizing array for euclidean distance

    # In euclidean distance i am not using square root because effect remains the same in both the cases 
    for k1 in range(K): 
        tempDist = np.sum((X-Centroids[:,k1])**2,axis=1) # Step by step using centriod and calculating difference sqaure for each example
        EuDis = np.c_[EuDis,tempDist] # Appending new column of distances in the calculated euclidean of the old one 
    C = np.argmin(EuDis,axis=1)+1 # Creating a index list which depicts which data sample belongs to which data sample
    Y ={} # Initiallizing a output dictionary which is in the form of {centriod1 : all member corresponding to that cluster)
    weight_c={} # Initiallizing a output dictionary which is in the form of {centriod1 : weight of that cluster)
    for k in range(K): # Creating four array as a value in the dictionary 
        Y[k+1] = np.array([]).reshape(4,0)
        weight_c[k+1] = []
    for i in range(m): # Appending all examples as per their corresponding centroids
        Y[C[i]] = np.c_[Y[C[i]],X[i]] 
        weight_c[C[i]].append(weights[i])
    for k in range(K): # Transposing the Y
        Y[k+1] = Y[k+1].T
        #print(weight_c[k+1])
    for k in range(K): # Calculating new centriod which is median out of all member present inside that cluster
        #calculated weighted median
        #Centroids[:,k] = np.median(Y[k+1],axis=0)
        temp=[]
        for i in range(Y[k+1].shape[0]):
            for _ in range(weight_c[k+1][i]):
                temp.append(Y[k+1][i]) 
        if(len(temp) != 0 ):     # CHANGED TO NOT EQUAL - MUSKAN
            Centroids[:,k] = np.median(temp,axis=0)

    # Repeat above explained process for n_iter number of times.
    for f in range(n_iter):
        EuDis = np.array([]).reshape(m,0)
        for k1 in range(K):
            tempDist = np.sum((X-Centroids[:,k1])**2,axis=1)
            EuDis = np.c_[EuDis,tempDist]
        C = np.argmin(EuDis,axis=1)+1
        Y ={}
        weight_c={}
        for k in range(K):
            Y[k+1] = np.array([]).reshape(4,0)
            weight_c[k+1] = []
        for i in range(m):
            Y[C[i]]=np.c_[Y[C[i]],X[i]]
            weight_c[C[i]].append(weights[i])
        for k in range(K):
            Y[k+1] = Y[k+1].T
        for k in range(K):
            #calculated weighted median by recreating the array (Note better formula would be to use weighter geometric median formula)
            temp=[]
            for i in range(Y[k+1].shape[0]):
                for _ in range(weight_c[k+1][i]):
                    temp.append(Y[k+1][i]) 
            if(len(temp) == 0):
                Centroids[:,k] = np.median(temp,axis=0)

    #print(Y)
    weight_out = [] #final weight for each representative
    for k in range(K):
        weight_out.append(int(np.sum(weight_c[k+1])))

    
    return Centroids.T,weight_out

# Stream Generator

In [0]:
def generate_X(X,m): #Generator function for sending data in the multiple of m 
    i=0 
    while True:
        yield X[i*m:(i+1)*m,:]
        i=i+1

# Main Algorithm

In [7]:
stream = generate_X(data_X,ms) # Creating Stream Generator Object
r = int(ms/K) # Choosing maximum value of Data Chunks that we can store
rcnt = [0] #Number of sets of K Centriods
level = 0 # For level Respresntative
centroids = [[]] #For Centriods
weights = [[]] # For Weights
counter=1 #chunk counter
while True: # Inifnite Loop

    if(level == 0): # When we have just started
        X = next(stream) # taking stream data
        if(X.shape[0] == 0): # if no rows are there, then dataset is empty or stream ended
            print('Stream ended')
            break
        print('D',counter) # Data chunk
        counter=counter+1 

        print('Calculating level ',level+1,' representatives...')
        Cen,weig = k_med(np.array(X),K) #Calculating Centeriods and their weights

        for k in range(K):
            centroids[level].append(Cen[k]) #Appending level-wise Centroids
            weights[level].append(weig[k]) # Appending level-wise weights
    
        rcnt[level] += 1 # Incrementing No. of set of Representatives
        print('Current Number of points at each level:',rcnt)

        if(rcnt[level] == r): # When No. of set of Representatives become r
            print('Increasing level',level+1,'-->',level+2) # We calulate next level representatives
            level +=1
            if(len(rcnt)<=level): # Initiallizing No. of Set of centriods, Centriods and weights for next level data
                rcnt.append(0)
                centroids.append([])
                weights.append([])
    else:
        X = centroids[level-1] # previous level Centriods
        weight_temp = weights[level-1] # previous level weights
        print('Calculating level ',level+1,' representatives...')
        Cen,weig = k_med(np.array(X),K,weights=weight_temp) # Calculating Centeriods and weights based on previous level centriods and weights
        
        for k in range(K): # Initiallizing No. of Set of centriods, Centriods and weights for next level data
            centroids[level].append(Cen[k])
            weights[level].append(weig[k])
    
        rcnt[level] += 1# Incrementing No. of set of Representatives

        if(rcnt[level] == r): # When No. of set of Representatives become r
            print('Increasing level',level+1,'-->',level+2)
            level +=1
            if(len(rcnt)<=level): # Initiallizing No. of Set of centriods, Centriods and weights for next level data
                rcnt.append(0)
                centroids.append([])
                weights.append([])
            #print('Current Number of points at each level:',rcnt)
        else:
            while(level!=0): # Reducing level and removing previous level representatives
                print('Decreasing level',level,'<--',level+1)
                print('Removing previous level representatives...')
                centroids[level-1] = []
                weights[level-1] = []
                rcnt[level-1] = 0
                level -=1
            print('Current Number of points at each level:',rcnt) 

remaining_centroids=[] # For remaining Centriods
remaining_weights=[] # For remaining Centriods
for level in range(len(rcnt)): # When at least one level have centriods 
    print('points at level',level,':',rcnt[level],len(centroids[level]),len(weights[level]))
    for cen,weig in zip(centroids[level],weights[level]): # Extrating the centriods and weights
        remaining_centroids.append(cen) # Remainig Centriods
        remaining_weights.append(weig) # Remainig Weights

final_centroids,final_weights = k_med(np.array(remaining_centroids),K,weights=remaining_weights) # Calculating k-median for remaing centriods and weights

D 1
Calculating level  1  representatives...
Current Number of points at each level: [1]
D 2
Calculating level  1  representatives...
Current Number of points at each level: [2]
D 3
Calculating level  1  representatives...
Current Number of points at each level: [3]
Stream ended
points at level 0 : 3 9 9


In [8]:
print(final_centroids,final_weights)

[[6.4 3.  5.3 2.1]
 [5.7 2.9 4.2 1.4]
 [5.  3.4 1.4 0.2]] [50, 50, 50]
